In [2]:
import os
import dart_fss as dart

corp_name = "NAVER"
# Open DART API KEY 설정
api_key=os.getenv("DART_API_KEY")
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

corp = corp_list.find_by_corp_name(corp_name, exactly=True)[0]

# 2022년부터 연간 연결재무제표 불러오기
fs = corp.extract_fs(bgn_de='20220101')

# 재무제표 검색 결과를 엑셀파일로 저장 ( 기본저장위치: 실행폴더/fsdata )
fs.save()

Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

'/Users/dudaji/gitRepo/dart/fsdata/00266961_annual.xlsx'